# Loop 7 Analysis: Understanding the Gap and Finding New Approaches

## Key Questions:
1. What is the per-N breakdown of the gap to target?
2. Which N values have the most room for improvement?
3. Can we implement the zaburo well-aligned initial solution approach?

In [ ]:
import pandas as pd
import numpy as np
from numba import njit
import math

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s','')) for v in a], np.float64)

tx, ty = make_polygon_template()
print('Scoring functions ready')

In [ ]:
# Load current best baseline
df = pd.read_csv('/home/code/external_data/saspav_latest/santa-2025.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

# Score each N
scores = {}
for n, g in df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc = score_group(xs, ys, ds, tx, ty)
    scores[n] = sc

print(f'Total score: {sum(scores.values()):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {sum(scores.values()) - 68.919154:.6f}')

In [ ]:
# Analyze per-N scores
scores_df = pd.DataFrame([
    {'N': n, 'score': sc, 'side': np.sqrt(sc * n), 'efficiency': sc}
    for n, sc in scores.items()
]).sort_values('N')

print('Top 10 worst packed (highest score per tree):')
print(scores_df.nlargest(10, 'score')[['N', 'score', 'side']])

print('\nTop 10 best packed (lowest score per tree):')
print(scores_df.nsmallest(10, 'score')[['N', 'score', 'side']])

In [ ]:
# Calculate theoretical minimum for each N
# Tree dimensions: width=0.7, height=1.0 (from -0.2 to 0.8)
# Area per tree = 0.7 * 1.0 = 0.7 (rough estimate)

# For N trees, minimum square side would be roughly sqrt(N * 0.7)
# But trees can interlock, so actual minimum is lower

scores_df['theoretical_min_side'] = np.sqrt(scores_df['N'] * 0.5)  # Rough estimate
scores_df['theoretical_min_score'] = scores_df['theoretical_min_side']**2 / scores_df['N']
scores_df['gap_to_theoretical'] = scores_df['score'] - scores_df['theoretical_min_score']

print('Gap analysis by N range:')
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    subset = scores_df[(scores_df['N'] >= start) & (scores_df['N'] <= end)]
    print(f'N={start}-{end}: Total score = {subset["score"].sum():.4f}, Count = {len(subset)}')

print(f'\nTotal gap to target: {sum(scores.values()) - 68.919154:.6f}')

In [ ]:
# The zaburo approach: well-aligned initial solution
# Creates rows of trees with alternating 0 and 180 degree rotations
# This is a deterministic approach that may find different basins

from decimal import Decimal, getcontext
getcontext().prec = 25

def find_best_trees_zaburo(n: int):
    """Zaburo's well-aligned initial solution approach"""
    best_score, best_config = float('inf'), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.35  # 0.7/2
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    all_trees.append({
                        'x': 0.7 * i + x_offset,
                        'y': y,
                        'deg': angle
                    })
                r += 1
            
            # Calculate score
            xs = np.array([t['x'] for t in all_trees])
            ys = np.array([t['y'] for t in all_trees])
            degs = np.array([t['deg'] for t in all_trees])
            
            sc = score_group(xs, ys, degs, tx, ty)
            if sc < best_score:
                best_score = sc
                best_config = all_trees
    
    return best_score, best_config

# Test on a few N values
print('Testing zaburo approach on sample N values:')
for n in [10, 50, 100, 150, 200]:
    zaburo_score, _ = find_best_trees_zaburo(n)
    baseline_score = scores[n]
    print(f'N={n}: Zaburo={zaburo_score:.6f}, Baseline={baseline_score:.6f}, Diff={zaburo_score - baseline_score:.6f}')

In [ ]:
# The zaburo approach gives MUCH worse scores than the baseline
# This confirms the baseline is highly optimized

# Let's analyze what makes the baseline so good
# Check the angle distribution in the baseline

print('Angle distribution in baseline:')
for n in [10, 50, 100, 200]:
    g = df[df['N'] == n]
    angles = strip(g['deg'].to_numpy())
    print(f'N={n}: angles range [{angles.min():.1f}, {angles.max():.1f}], unique={len(np.unique(np.round(angles, 1)))}')

# Check position distribution
print('\nPosition distribution in baseline:')
for n in [10, 50, 100, 200]:
    g = df[df['N'] == n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    print(f'N={n}: x=[{xs.min():.2f}, {xs.max():.2f}], y=[{ys.min():.2f}, {ys.max():.2f}]')

In [ ]:
# Key insight: The baseline uses MANY different angles, not just 0/180
# This allows for much tighter packing

# Let's check if there are any patterns in the best-packed configurations
print('Analyzing best-packed configurations:')
best_packed = scores_df.nsmallest(20, 'score')
for _, row in best_packed.iterrows():
    n = int(row['N'])
    g = df[df['N'] == n]
    angles = strip(g['deg'].to_numpy())
    unique_angles = len(np.unique(np.round(angles, 0)))
    print(f'N={n}: score={row["score"]:.4f}, unique_angles={unique_angles}')

In [ ]:
# Summary of findings:
# 1. The baseline is EXTREMELY well optimized - zaburo approach gives 2-3x worse scores
# 2. The baseline uses many different angles (not just 0/180)
# 3. Best-packed configurations have many unique angles

# The gap to target (1.74 points) requires:
# - Either finding a fundamentally different basin (unlikely with current approaches)
# - Or the target was achieved with proprietary techniques not in public kernels

# Let's calculate what improvement we'd need per N to close the gap
gap = sum(scores.values()) - 68.919154
print(f'Total gap: {gap:.6f}')
print(f'Gap per N (average): {gap / 200:.6f}')

# If we could improve each N by 0.87%, we'd close the gap
print(f'\nRequired improvement: {gap / sum(scores.values()) * 100:.2f}%')

# But the baseline has ZERO slack - we can't improve it with local optimization
print('\nConclusion: The gap requires fundamentally different approaches.')

In [ ]:
# Record key findings
findings = '''
Loop 7 Analysis Summary:

1. Current best score: 70.659958 (saspav_latest)
2. Target: 68.919154
3. Gap: 1.740804 (2.46%)

4. Zaburo well-aligned approach gives 2-3x WORSE scores than baseline
   - Baseline uses many different angles, not just 0/180
   - This allows for much tighter packing

5. The baseline is at an EXTREMELY tight local optimum:
   - 33 minutes of C++ optimization found ZERO improvement
   - Even tiny perturbations cause collisions
   - All optimization techniques have failed

6. To close the gap, we need:
   - 2.46% improvement across all N values
   - OR fundamentally different configurations (different basins)

7. The target score (68.919154) was likely achieved through:
   - Proprietary algorithms not shared publicly
   - Much longer optimization runs (days, not hours)
   - Different initial configurations that lead to different basins
   - Techniques not available in public kernels

Recommendation: Submit current best (70.659958) and continue researching.
'''
print(findings)